In [13]:
import os
import json

#pretrained_model = "Qwen/Qwen2.5-7B-Instruct"
pretrained_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

dataset = "MATH_train"



outputs_name =  pretrained_model.replace("/", ".") + "-" + dataset

os.makedirs(os.path.dirname("./temp_data/outputs-rl-" + outputs_name + ".json"), exist_ok=True)
with open("./temp_data/outputs-rl-" + outputs_name + ".json", 'r') as f:
    data = json.load(f)


max_generation_token = 10000

In [14]:
index_list = []
extracted_output_list = []
ground_truth_list = []
for i in range(len(data)):
    data[i]["correctness"] = []
    index_list = index_list + [i] * len(data[i]["extracted_output"])
    extracted_output_list = extracted_output_list + data[i]["extracted_output"]
    ground_truth_list = ground_truth_list + [data[i]["ground_truth_answer"]] * len(data[i]["extracted_output"])

In [15]:
from concurrent.futures import ThreadPoolExecutor
import math_utils
import asyncio
import nest_asyncio
nest_asyncio.apply()


async def get_correctness():
    executor = ThreadPoolExecutor(max_workers=64)
    tasks = []
    for i in range(len(index_list)):
        tasks.append(math_utils.is_equal(extracted_output_list[i], ground_truth_list[i], executor))
    results = await asyncio.gather(*tasks)
    return results


correctness_list = asyncio.run(get_correctness())

In [16]:
for i in range(len(index_list)):
    index_i = index_list[i]
    data[index_i]["correctness"].append(correctness_list[i])

In [17]:
import numpy as np
from scipy.stats import norm

def compute_reversed_ranks(correct: list[bool], lengths: list[int]) -> list[int]:
    n = len(correct)
    items = list(enumerate(zip(correct, lengths)))
    items_sorted = sorted(
        items,
        key=lambda x: (-int(x[1][0]), x[1][1])
    )
    ranks = [0] * n
    for i, (idx, _) in enumerate(items_sorted):
        ranks[idx] = n - i
    return ranks

In [23]:
i = 0
correctness = data[i]["correctness"]
lengths = data[i]["response_length"]
rewards = [correctness[i] + 0.001 * lengths[i] for i in range(len(correctness))]

In [ ]:
final_data = []
for i in range(len(data)):
    correctness = data[i]["correctness"]
    lengths = data[i]["response_length"]
    #ranks = compute_reversed_ranks(correctness, lengths)
    #probs  = np.array(ranks) / (len(ranks) + 1)
    #rewards = norm.ppf(probs)
    data[i]["rewards"] = rewards

    for j in range(len(rewards)):
        data_i = {}
        data_i["prompt"] = data[i]["prompt"]
        data_i["reward"] = rewards[j]
        if lengths[j] < max_generation_token:
            data_i["response"] = data[i]["full_output"][j] + "<|im_end|>"
        else:
            data_i["response"] = data[i]["full_output"][j]
        final_data.append(data_i)

In [23]:
final_data[0]

{'prompt': '<|im_start|>You are a helpful assistant help user solve problems. <|im_end|>\n<|im_start|>user \nYou need to put your final answer in \\boxed{}. This is the problem:\nThe function $f$ is not defined for $x = 0,$ but for all non-zero real numbers $x,$\n\\[f(x) + 2f \\left( \\frac{1}{x} \\right) = 3x.\\]Find the real solutions to $f(x) = f(-x).$  Enter the real solutions, separated by commas. <|im_end|>\n<|im_start|>assistant ',
 'reward': 0.7215222839823432,
 'response': '1. We start with the given functional equation:\n   \\[\n   f(x) + 2f \\left( \\frac{1}{x} \\right) = 3x\n   \\]\n2. Substitute \\( x \\) with \\( \\frac{1}{x} \\):\n   \\[\n   f \\left( \\frac{1}{x} \\right) + 2f(x) = \\frac{3}{x}\n   \\]\n\n3. We now have a system of linear equations:\n   \\[\n   \\begin{cases}\n   f(x) + 2f \\left( \\frac{1}{x} \\right) = 3x \\\\\n   2f(x) + f \\left( \\frac{1}{x} \\right) = \\frac{3}{x}\n   \\end{cases}\n   \\]\n\n4. To eliminate \\( f \\left( \\frac{1}{x} \\right) \\),

In [8]:
import nest_asyncio